In [ ]:
# MIT License
# 
# Copyright (c) 2025 Danielle N. Alverson,
# Eric Fonseca, Kausturi Parui, Steph J. Meikle
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# -------------------------------------------------------------
# Imports (Dash removed; replaced with ipywidgets + Plotly)
# -------------------------------------------------------------

import pandas as pd
import numpy as np
import plotly.express as px
import io
import os
import base64
import tempfile

# wrpoly behaves the same as in the Dash app
import wrpoly as wrp

import ipywidgets as widgets
from IPython.display import display, clear_output, Image, Markdown


In [ ]:
# -------------------------------------------------------------
# Load Data
# (Matches Dash version: merged_df + unique_df)
# -------------------------------------------------------------

merged_df = pd.read_csv('merged_cif_data_0425.csv', index_col=0)

# Remove duplicate CIFs and keep first occurrence
unique_df = merged_df.drop_duplicates(subset='cif_name', keep='first').copy()

# -------------------------------------------------------------
# Column renaming (same dictionary as original Dash application)
# -------------------------------------------------------------

other_new_names = {
    'bond_angle_variance': 'Bond Angle Variance',
    'Structure type': 'Structure Type',
    'Average Bond Angles': 'Average Bond Length Angles',
    'Octahedral Distortion?': 'Octahedral Distortion'
}

new_column_names = {
    'avg_bond_length': 'Average Bond Length',
    'std_bond_length': 'Standard Deviation Bond Length',
    'skew_bond_length': 'Skew Bond Length',
    'distortion_index': 'Distortion Index',
    'quadratic_elongation': 'Quadratic Elongation',
    'n_corner_pairs': 'Number of Corner Pairs',
    'n_edge_pairs': 'Number of Edge Pairs',
    'bond_angle_variance': 'Bond Angle Variance',
    'n_face_pairs': 'Number of Face Pairs',
    'n_atoms_in_cell': 'Number of Atoms in Unit Cell',
    'avg_bond_angles': 'Average Bond Angles',
    'std_bond_length_angles': 'Standard Deviation Bond Length Angles',
    'skew_bond_length_angles': 'Skew Bond Length Angles',
    'volumes': 'Volumes',
    'central_atoms': 'Central Atoms',
    'polyhedra_formula': 'Polyhedra Formula',
    'poly_types': 'Polyhedra Types',
    'n_atoms_per_cell': 'Number of Atoms per Unit Cell',
    'cif_name': 'Cif Name'
}

# Apply renaming to both dataframes
merged_df.rename(columns=other_new_names, inplace=True)
unique_df.rename(columns=other_new_names, inplace=True)
merged_df.rename(columns=new_column_names, inplace=True)
unique_df.rename(columns=new_column_names, inplace=True)

# -------------------------------------------------------------
# Float coercion – identical logic to Dash app version
# (convert columns to float, convert errors to NaN -> then fill with 0)
# -------------------------------------------------------------

float_columns = [
    'Average Bond Length', 'Standard Deviation Bond Length', 'Skew Bond Length',
    'Distortion Index', 'Quadratic Elongation', 'Number of Corner Pairs',
    'Number of Edge Pairs', 'Bond Angle Variance', 'Number of Face Pairs',
    'Number of Atoms in Unit Cell', 'Average Bond Length Angles',
    'Standard Deviation Bond Length Angles', 'Skew Bond Length Angles',
    'Volumes', 'Central Atoms', 'Polyhedra Formula', 'Polyhedra Types',
    'Number of Atoms per Unit Cell', 'Capacity (mAh/g)', '1st Charge (mAh/g)',
    '1st Discharge (mAh/g)', '2nd Discharge (mAh/g)', 'Mol wt (g/mol)',
    'Total Number of TM', 'Li/TM ', 'x moles of Li+',
    'Edge pairs/Corner Pairs', '1st Charge/1st Discharge Ratio',
    '2nd/1st Discharge Ratio'
]

for col in float_columns:
    if col in merged_df.columns:
        merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').fillna(0)
    if col in unique_df.columns:
        unique_df[col] = pd.to_numeric(unique_df[col], errors='coerce').fillna(0)

# Store list of columns for dropdowns
all_columns = list(merged_df.columns)

print(f"Loaded merged_df with {merged_df.shape[0]} rows and {merged_df.shape[1]} columns.")


In [ ]:
# -------------------------------------------------------------
# Create widgets to replace Dash dropdowns + inputs
# -------------------------------------------------------------

# Material type dropdown (replicates Dash structure-selection menu)
material_type_dropdown = widgets.Dropdown(
    options=[
        ('All Materials', 'all'),
        ('Wadsley-Roth Materials', 'Wadsley-Roth'),
        ('Wadsley-Roth Adjacent Materials', 'Wadsley-Roth Adjacent')
    ],
    value='all',
    description='Material type:',
    layout=widgets.Layout(width='300px')
)

# Common list of selectable columns
column_options = [(col, col) for col in all_columns]

# The following dropdowns match Dash equivalents
x_dropdown = widgets.Dropdown(
    options=column_options,
    value='Distortion Index' if 'Distortion Index' in merged_df.columns else all_columns[0],
    description='X:',
    layout=widgets.Layout(width='300px')
)

y_dropdown = widgets.Dropdown(
    options=column_options,
    value='Capacity (mAh/g)' if 'Capacity (mAh/g)' in merged_df.columns else all_columns[1],
    description='Y:',
    layout=widgets.Layout(width='300px')
)

size_dropdown = widgets.Dropdown(
    options=[('None', None)] + column_options,
    value='Volumes' if 'Volumes' in merged_df.columns else None,
    description='Size:',
    layout=widgets.Layout(width='300px')
)

color_dropdown = widgets.Dropdown(
    options=[('None', None)] + column_options,
    value='Formula' if 'Formula' in merged_df.columns else None,
    description='Color:',
    layout=widgets.Layout(width='300px')
)

shape_dropdown = widgets.Dropdown(
    options=[('None', None)] + column_options,
    value='Polyhedra Types' if 'Polyhedra Types' in merged_df.columns else None,
    description='Shape:',
    layout=widgets.Layout(width='300px')
)

legend_checkbox = widgets.Checkbox(
    value=False,
    description='Show legend'
)

# Search bar (replicates the "Search Formula" Dash input)
search_bar = widgets.Text(
    value='',
    placeholder='Enter formula or CIF name',
    description='Search:',
    layout=widgets.Layout(width='300px')
)

# File upload (replacement for Dash Upload component)
upload = widgets.FileUpload(
    accept='.cif',
    multiple=False,
    description='Upload CIF'
)

# “I believe in fairies” button – same message as original
fairies_button = widgets.Button(
    description='I believe in fairies',
    tooltip='Click ONCE after importing CIF file',
    layout=widgets.Layout(width='300px')
)

status_label = widgets.HTML(value='')

# Output area for the scatter plot
output_fig = widgets.Output()


In [ ]:
# -------------------------------------------------------------
# Helpers for CIF upload + filtering (converted from Dash callbacks)
# -------------------------------------------------------------

def get_upload_info():
    """
    Normalize access to ipywidgets FileUpload data.
    Handles multiple widget versions.
    Returns (filename, bytes_content).
    """
    if not upload.value:
        return None, None

    val = upload.value

    # ipywidgets ≥ 8 uses dict; older versions used tuple
    if isinstance(val, dict):
        fileinfo = next(iter(val.values()))
    elif isinstance(val, (tuple, list)):
        fileinfo = val[0]
    else:
        return None, None

    return fileinfo['metadata']['name'], fileinfo['content']


def append_uploaded_cif(df_base):
    """
    Process uploaded CIF file with wrpoly (as in Dash app),
    append to dataframe, return updated dataframe + status message.
    """
    filename, content = get_upload_info()
    if filename is None:
        return df_base, ''

    # Save uploaded file to temp path (matching Dash logic)
    with tempfile.NamedTemporaryFile(suffix='.cif', delete=False) as tmp:
        tmp.write(content)
        temp_path = tmp.name

    try:
        cif_name = os.path.splitext(filename)[0]

        # wrpoly: load structure + compute averages
        structure = wrp.get_structure(temp_path)
        uploaded_df = wrp.get_average_df(structure, cif_name)

        # Apply renaming dictionary (from Dash app)
        uploaded_df.rename(columns=new_column_names, inplace=True)

        combined_df = pd.concat([df_base, uploaded_df], ignore_index=True)
        status_msg = f'File "{filename}" uploaded and processed.'

    except Exception as e:
        combined_df = df_base
        status_msg = f'Error processing "{filename}": {e}'

    finally:
        try:
            os.remove(temp_path)
        except:
            pass

    # Clear widget to prevent repeated re-adding on interaction:
    try:
        upload.value.clear()
    except:
        try:
            upload.value = ()
        except:
            pass

    return combined_df, status_msg


def make_filtered_dataframe():
    """
    Recreates Dash callback behavior:
      - Filter by material type
      - Filter by search term
      - Append uploaded CIF file
    """
    wr_mat = material_type_dropdown.value

    # Material selection filter
    if wr_mat == 'all':
        df = merged_df.copy()
    elif wr_mat == 'Wadsley-Roth':
        df = merged_df[merged_df['Structure'] == 'Wadsley-Roth'].copy()
    elif wr_mat == 'Wadsley-Roth Adjacent':
        df = merged_df[merged_df['Structure'] == 'Wadsley-Roth Adjacent'].copy()
    else:
        df = merged_df.copy()

    # Search filter
    term = search_bar.value.strip().lower()
    if term:
        mask = False
        if 'Cif Name' in df.columns:
            mask = mask | df['Cif Name'].astype(str).str.lower().str.contains(term)
        if 'Formula' in df.columns:
            mask = mask | df['Formula'].astype(str).str.lower().str.contains(term)
        df = df[mask].copy()

    # Uploaded CIF appended
    df, status_msg = append_uploaded_cif(df)

    return df, status_msg


In [ ]:
# -------------------------------------------------------------
# Update plot whenever widgets change (converted from Dash callback)
# -------------------------------------------------------------

def update_plot(*args):
    """
    Recreates Dash scatter-plot callback.
    Called whenever dropdowns, search bar, or CIF upload changes.
    """
    with output_fig:
        clear_output(wait=True)

        df, status_msg = make_filtered_dataframe()

        # Get selected parameters
        x = x_dropdown.value
        y = y_dropdown.value
        size = size_dropdown.value
        color = color_dropdown.value
        symbol = shape_dropdown.value

        # Validate axes
        if x not in df.columns or y not in df.columns:
            print("Selected X or Y not found in dataframe.")
            status_label.value = status_msg
            return

        # Optional parameters need None fallback
        size_arg = size if (size in df.columns) else None
        color_arg = color if (color in df.columns) else None
        symbol_arg = symbol if (symbol in df.columns) else None

        # Hover info
        hover_cols = [col for col in ['Cif Name', 'Formula'] if col in df.columns]

        # Create figure (same as Dash px.scatter)
        fig = px.scatter(
            df,
            x=x,
            y=y,
            size=size_arg,
            color=color_arg,
            symbol=symbol_arg,
            hover_data=hover_cols
        )

        # Layout styling (copied from Dash)
        fig.update_layout(
            font_family="Helvetica",
            font_color="black",
            title_font_family="Helvetica",
            title_font_color="black",
            template='presentation',
            width=900,
            height=600,
            title=f"{x} vs. {y}",
            title_x=0.5,
            margin=dict(l=70, r=70, t=70, b=70),
            showlegend=legend_checkbox.value
        )

        fig.show()
        status_label.value = status_msg


def on_fairies_clicked(button):
    """
    Replacement for the Dash "Click ONCE after importing CIF" helper button.
    Simply forces a manual plot update.
    """
    update_plot()


# Wire widget events → update plot
widgets_to_watch = [
    material_type_dropdown,
    x_dropdown,
    y_dropdown,
    size_dropdown,
    color_dropdown,
    shape_dropdown,
    legend_checkbox,
    search_bar,
    upload
]

for w in widgets_to_watch:
    w.observe(update_plot, names='value')

fairies_button.on_click(on_fairies_clicked)

# UI assembly
ui_left = widgets.VBox([
    material_type_dropdown,
    x_dropdown,
    y_dropdown,
    size_dropdown,
    color_dropdown,
    shape_dropdown,
    legend_checkbox,
    search_bar,
    upload,
    fairies_button,
    status_label
])

ui = widgets.HBox([ui_left, output_fig])

# Display interface
display(ui)

# Initial plot
update_plot()


In [ ]:
# -------------------------------------------------------------
# Footer: NSF logo + grant number + authors (copied from Dash)
# -------------------------------------------------------------

# Replace with your path if needed
nsf_logo_path = "NSF_Official_logo_High_Res_1200ppi.png"

try:
    display(Image(filename=nsf_logo_path, width=150))
except:
    print(f"NSF logo file not found at {nsf_logo_path}")

display(Markdown("""
**NSF DMR-2334240**

Authors: Danielle N. Alverson, Kausturi Parui, Eric Fonseca,  
Steph J. Meikle, Megan M. Butala  

This project is licensed under the MIT License.
"""))
